In [ ]:
import numpy as np
import MFCC
import gmmhmm

In [ ]:
A = np.array([[.65, .35], [.15, .85]])
pi = np.array([.8, .2])
weights = np.array([[.7, .2, .1], [.1, .5, .4]])
means1 = np.array([[0., 17., -4.], [5., -12., -8.], [-16., 22., 2.]])
means2 = np.array([[-5., 3., 23.], [-12., -2., 14.], [15., -32., 0.]])
means = np.array([means1, means2])
covars1 = np.array([5*np.eye(3), 7*np.eye(3), np.eye(3)])
covars2 = np.array([10*np.eye(3), 3*np.eye(3), 4*np.eye(3)])
covars = np.array([covars1, covars2])
gmmhmm = [A, weights, means, covars, pi]
sample_component = np.argmax(np.random.multinomial(1, weights[1,:]))
sample = np.random.multivariate_normal(means[1, sample_component, :], -covars[1, sample_component, :, :])

### Problem 1

In [ ]:
def sample_gmmhmm(gmmhmm, n_sim):
    """
    Simulate sampling from a GMMHMM.
    Returns
    -------
    states : ndarray of shape (n_sim,)
    The sequence of states
    obs : ndarray of shape (n_sim, K)
    The generated observations (column vectors of length K)
    """
    weights = gmmhmm[1]
    means = gmmhmm[2]
    covars = gmmhmm[3]
    N = weights.shape[0]
    states = [np.random.randint(0,N)]
    K = weights.shape[1]
    obs = np.zeros((n_sim,))
    for i in range(n_sim) :
        sample_component = np.argmax(np.random.multinomial(states[i], weights[1,:]))
        sample = np.random.multivariate_normal(means[states[i], sample_component, :], -covars[states[i], sample_component, :, :])
        obs[i,:] = sample
        states.append(sample.index(np.argmax(sample)))
    return np.array(states), obs

### Problem 2

In [ ]:
# assume sample is an array of length 88200
mfccs = MFCC.extract(sample)

In [ ]:
startprob, transmat = initialize(5)
model = gmmhmm.GMMHMM(n_components=5, n_mix=3, transmat=transmat, startprob=startprob, cvtype='diag')
# these values for covars_prior and var should work well for this problem
model.covars_prior = 0.01
model.fit(samples, init_params='mc', var=0.1)
print model.logprob

### Problem 3

In [ ]:
score = model.score(obs)

### Problem 4